In [1]:
# -*- coding: utf-8 -*-
"""
Analyse des capacités de productions d'électricité en métropole  française

Created on Wed May 29 13:54:11 2024

@author: Thierry ALLEM
"""

"\nAnalyse des capacités de productions d'électricité en métropole  française\n\nCreated on Wed May 29 13:54:11 2024\n\n@author: Thierry ALLEM\n"

In [2]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Chargement du fichier du registre national des installations de production et stockage d'électricité
df_cap_prod = pd.read_csv('registre_national_installation_production_stockage_electricite.csv', sep=';',low_memory=False)
df_cap_prod.head()

,nomInstallation,codeEICResourceObject,codeIRIS,codeINSEECommune,commune,codeEPCI,EPCI,codeDepartement,departement,codeRegion,...,capaciteReservoir,hauteurChute,productible,debitMaximal,codeGestionnaire,gestionnaire,energieAnnuelleGlissanteInjectee,energieAnnuelleGlissanteProduite,energieAnnuelleGlissanteSoutiree,energieAnnuelleGlissanteStockee
0,Confidentiel,17W000000034442R,2A0010000,2A001,Afa,242010056,CA du Pays Ajaccien,2A,Corse-du-Sud,94.0,...,NaN,NaN,NaN,NaN,23X160203-000021,EDF-SEI,0.0,NaN,0.0,NaN
1,Confidentiel,17W0000000383544,2A0010000,2A001,Afa,242010056,CA du Pays Ajaccien,2A,Corse-du-Sud,94.0,...,NaN,NaN,NaN,NaN,23X160203-000021,EDF-SEI,0.0,NaN,0.0,NaN
2,Confidentiel,17W000000032988U,2A0010000,2A001,Afa,242010056,CA du Pays Ajaccien,2A,Corse-du-Sud,94.0,...,NaN,NaN,NaN,NaN,23X160203-000021,EDF-SEI,0.0,NaN,0.0,NaN
3,Confidentiel,17W000001336047V,2A0010000,2A001,Afa,242010056,CA du Pays Ajaccien,2A,Corse-du-Sud,94.0,...,NaN,NaN,NaN,NaN,23X160203-000021,EDF-SEI,0.0,NaN,0.0,NaN
4,Confidentiel,17W000001336508L,2A0010000,2A001,Afa,242010056,CA du Pays Ajaccien,2A,Corse-du-Sud,94.0,...,NaN,NaN,NaN,NaN,23X160203-000021,EDF-SEI,0.0,NaN,0.0,NaN


In [4]:
# Export des 100 premières lignes dans un nouveau fichier CSV pour aperçu
df_cap_prod_head = df_cap_prod.head(100)
df_cap_prod_head.to_csv('registre_national_installation_production_first_100_lines.csv', sep=';', encoding='latin-1', index=False)

In [5]:
# Affichage du nombre de lignes et colonnes du dataframe et décompte des types des valeurs

print("Nombre de lignes du dataframe : ", len(df_cap_prod.index))
print("Nombre de colonnes du dataframe : ", len(df_cap_prod.columns))
print("Valeurs uniques de la colonne 'region' :", df_cap_prod['region'].unique())
print("Types de valeurs :",df_cap_prod.dtypes.value_counts() )
df_cap_prod.info()


Nombre de lignes du dataframe :  73693
Nombre de colonnes du dataframe :  49
Valeurs uniques de la colonne 'region' : ['Corse' 'Auvergne-Rhône-Alpes' 'Hauts-de-France'
 "Provence-Alpes-Côte d'Azur" 'Grand Est' 'Occitanie' 'Normandie'
 'Nouvelle-Aquitaine' 'Centre-Val de Loire' 'Bourgogne-Franche-Comté'
 'Bretagne' 'Pays de la Loire' 'Île-de-France' 'Guadeloupe' 'Martinique'
 'Guyane' 'La Réunion' nan]
Types de valeurs : object     32
float64    16
int64       1
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73693 entries, 0 to 73692
Data columns (total 49 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   nomInstallation                   73693 non-null  object 
 1   codeEICResourceObject             52279 non-null  object 
 2   codeIRIS                          43125 non-null  object 
 3   codeINSEECommune                  72397 non-null  object 
 4   commune         

In [6]:
#   Renommage des colonnes

#     Dictionnaire avec les nouveaux noms des colonnes

dictionnaire_entetes = {'nomInstallation ':'nom_installation',
                        'codeEICResourceObject':'code_EIC',
                        'codeRegion' : 'code_region',                
                'dateRaccordement' : 'date_raccordement',
                'dateDeraccordement' : 'date_deraccordement',
                'dateMiseEnService' : 'date_mise_en_service',
                'codeFiliere' :'code_filiere',
                'filiere' : 'filiere_production',
                'codeCombustible' : 'code_combustible',
                'codesCombustiblesSecondaires' : 'codes_combustibles_secondaires',
                'combustiblesSecondaires' : 'combustibles_secondaires',
                'codeTechnologie' : 'code_technologie',
                'typeStockage' : 'type_stockage',
                'puisMaxInstallee' : 'puis_max_installee',
                'puisMaxRacCharge' : 'puis_max_rac_charge',
                'puisMaxCharge' : 'puis_max_charge',
                'puisMaxRac' : 'puis_max_rac',
                'puisMaxInstalleeDisCharge' : 'puis_max_installee_discharge',
                'energieStockable' : 'energie_stockable',
                'energieAnnuelleGlissanteInjectee' : 'energie_annuelle_glissante_injectee',
                'energieAnnuelleGlissanteProduite' : 'energie_annuelle_glissante_produite',
                'energieAnnuelleGlissanteSoutiree' : 'energie_annuelle_glissante_soutiree',
                'energieAnnuelleGlissanteStockee' : 'energie_annuelle_glissante_stockee'}
                
df_cap_prod=df_cap_prod.rename(dictionnaire_entetes, axis = 1)

In [7]:
# DESCRIPTION DES COLONNES
#    Création d'un dataframe de description des colonnes de df_cap_prod, sans regroupement par region

df_colonnes = pd.DataFrame()

#   Stockage du noms des colonnes du  dataframe df_cap_prod
df_colonnes['Nom_colonne'] = list(df_cap_prod.columns)

#   Affichage du type des valeurs des colonnes de df
df_colonnes.index = df_cap_prod.columns
types_valeurs = df_cap_prod.dtypes
df_colonnes['Type_valeurs'] = types_valeurs

#   Affichage des valeurs uniques

df_colonnes.insert(loc=len(df_colonnes.columns), column='Valeurs_uniques', value='Valeurs continues')

df_colonnes.loc[(df_colonnes['Nom_colonne'] =='code_region'),'Valeurs_uniques'] = str(df_cap_prod['code_region'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='region'),'Valeurs_uniques'] = str(df_cap_prod['region'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='code_filiere'),'Valeurs_uniques'] = str(df_cap_prod['code_filiere'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='filiere_production'),'Valeurs_uniques'] = str(df_cap_prod['filiere_production'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='code_combustible'),'Valeurs_uniques'] = str(df_cap_prod['code_combustible'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='combustible'),'Valeurs_uniques'] = str(df_cap_prod['combustible'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='combustibles_secondaires'),'Valeurs_uniques'] = str(df_cap_prod['combustibles_secondaires'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='code_technologie'),'Valeurs_uniques'] = str(df_cap_prod['code_technologie'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='technologie'),'Valeurs_uniques'] = str(df_cap_prod['technologie'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='type_stockage'),'Valeurs_uniques'] = str(df_cap_prod['type_stockage'].unique())

pd.set_option('display.max_colwidth', None)
#   Affichage du nombre de valeurs des colonnes de df
df_colonnes['Nb_valeurs'] = df_cap_prod.count()

#   Affichage des NaN

#        Affichage du nombre de NaN de chaque colonne
df_colonnes['nb_NaN'] = df_cap_prod.isna().sum()

#        Quantité totale de 'valeurs' + 'NaN'
df_colonnes['Nb_datas'] = df_colonnes['Nb_valeurs']+df_colonnes['nb_NaN']

#       Affichage du % de données manquantes
df_colonnes['%_Valeurs'] = round(df_colonnes['Nb_valeurs']/df_colonnes['Nb_datas']*100,2)
df_colonnes['%_NaN'] =round(df_cap_prod.isna().sum() /df_colonnes['Nb_datas']*100,2)


#   Export du compte-rendu des colonnes dans un fichier Excel

file_export2 ='2_Rapp_explo_col_registre_prod.xlsx'

df_colonnes.to_excel(file_export2)

print("Rapport préliminaire d'exploration des colonnes du dataframe 'registre_national_installation_production_stockage_electricite' exporté avec succès dans le fichier Excel '2_Rapp_explo_col_registre_prod.xlsx'")

Rapport préliminaire d'exploration des colonnes du dataframe 'registre_national_installation_production_stockage_electricite' exporté avec succès dans le fichier Excel '2_Rapp_explo_col_registre_prod.xlsx'


In [8]:
# Suppression des lignes associées à des régions non traitées dans le projet Eco2Mix
regions_to_remove = ['Corse', 'Guadeloupe', 'Martinique', 'Guyane', 'La Réunion']
df_cap_prod = df_cap_prod[df_cap_prod['region'].isin(regions_to_remove) ==False]


In [9]:
# Suppression des lignes avec NaN dans la colonne 'region' (selon les codes INSEE communes, ces NaN semblent correspondre à
# des installations situées dans les DOM-TOM; on les suppose donc hors champs du projet)
df_cap_prod = df_cap_prod.dropna(subset=['region'])

In [10]:
print(df_cap_prod.columns.tolist())

['nomInstallation', 'code_EIC', 'codeIRIS', 'codeINSEECommune', 'commune', 'codeEPCI', 'EPCI', 'codeDepartement', 'departement', 'code_region', 'region', 'codeIRISCommuneImplantation', 'codeINSEECommuneImplantation', 'codeS3RENR', 'date_raccordement', 'date_deraccordement', 'date_mise_en_service', 'dateDebutVersion', 'posteSource', 'tensionRaccordement', 'modeRaccordement', 'code_filiere', 'filiere_production', 'code_combustible', 'combustible', 'codes_combustibles_secondaires', 'combustibles_secondaires', 'code_technologie', 'technologie', 'type_stockage', 'puis_max_installee', 'puis_max_rac_charge', 'puis_max_charge', 'puis_max_rac', 'puis_max_installee_discharge', 'nbGroupes', 'nbInstallations', 'regime', 'energie_stockable', 'capaciteReservoir', 'hauteurChute', 'productible', 'debitMaximal', 'codeGestionnaire', 'gestionnaire', 'energie_annuelle_glissante_injectee', 'energie_annuelle_glissante_produite', 'energie_annuelle_glissante_soutiree', 'energie_annuelle_glissante_stockee']


In [11]:
# TRAITEMENT DES DOUBLONS
print("Nombre de doublons dans la colonne 'code_EIC' (identification de chaque installation):", df_cap_prod['code_EIC'].duplicated().sum())
print("Nombre de doublons dans la totalité du dataframe :", df_cap_prod.duplicated().sum())
#       Marquage des lignes en double
duplicates_mark = df_cap_prod.duplicated(subset=['code_EIC'], keep=False)

#       Création d'un nouveau dataframe avec les lignes en double
df_duplicates = df_cap_prod[duplicates_mark]
print(df_duplicates.head())
print("Nombre de doublons du dataframe filtré:",df_duplicates.duplicated().sum())

#       Export du dataframe des doublons dans un fichier Excel

file_export1 ='1_Doublons_installations.xlsx'

df_duplicates.to_excel(file_export1)

print("Doublons des installations du dataframe 'registre_national_installation_production_stockage_electricite' exporté avec succès dans le fichier Excel 1_Doublons_installations.xlsx'")

Nombre de doublons dans la colonne 'code_EIC' (identification de chaque installation): 21273
Nombre de doublons dans la totalité du dataframe : 0
                                   nomInstallation code_EIC   codeIRIS  \
712  Agrégation des installations de moins de 36KW      NaN  010010000   
713  Agrégation des installations de moins de 36KW      NaN  010040101   
715  Agrégation des installations de moins de 36KW      NaN  010040202   
717  Agrégation des installations de moins de 36KW      NaN  010040102   
718  Agrégation des installations de moins de 36KW      NaN        NaN   

    codeINSEECommune                  commune   codeEPCI  \
712            01001  L'Abergement-Clémenciat  200069193   
713            01004        Ambérieu-en-Bugey  240100883   
715            01004        Ambérieu-en-Bugey  240100883   
717            01004        Ambérieu-en-Bugey  240100883   
718            01004        Ambérieu-en-Bugey  240100883   

                         EPCI codeDepartement de

In [12]:
# TRAITEMENT DES DONNEES DU REGISTRE df_cap_prod

In [13]:
#   Suppression des colonnes non utiles au projet, notamment celles précisant les localisations; la présence de doublons ayant été exclue
colonnes_to_drop = [2, 3, 4, 5, 6, 7, 8,  11, 12, 13, 17, 18, 19, 20, 35, 36, 37, 39, 40, 42, 43, 44]
df_cap_prod.drop(columns = df_cap_prod.columns[colonnes_to_drop], inplace =True)

In [14]:
#   Copie du dataframe avant pre-processing
df_cap_prod2 = df_cap_prod.copy()

In [15]:
# GESTION DES DATES

In [16]:
#   Remplacement de valeurs des colonnes 'date' inscrit dans un format différent et avec d'autres séparateurs des autres valeurs
liste_dates_replace = {'1893-01-01':'01/01/1893', '1897-03-21':'21/03/1897','1898-01-01':'01/01/1898'}
df_cap_prod2.replace(liste_dates_replace, inplace =True)

In [17]:
#   Conversion des formats de date
colonnes_a_convertir = ['date_raccordement','date_deraccordement','date_mise_en_service']
for colonne in colonnes_a_convertir:
    df_cap_prod2[colonne] = pd.to_datetime(df_cap_prod2[colonne], format="%d/%m/%Y", dayfirst=True).dt.strftime("%Y-%m-%d")
    df_cap_prod2[colonne] = pd.to_datetime(df_cap_prod2[colonne])
df_cap_prod2.dtypes

nomInstallation                                object
code_EIC                                       object
code_region                                   float64
region                                         object
date_raccordement                      datetime64[ns]
date_deraccordement                    datetime64[ns]
date_mise_en_service                   datetime64[ns]
code_filiere                                   object
filiere_production                             object
code_combustible                               object
combustible                                    object
codes_combustibles_secondaires                 object
combustibles_secondaires                       object
code_technologie                               object
technologie                                    object
type_stockage                                  object
puis_max_installee                            float64
puis_max_rac_charge                           float64
puis_max_charge             

In [18]:
#   Gestion des valeurs manquantes des colonnes de dates  ****************************
#       Colonne 'date_raccordement': si non-renseignée, on suppose que l'installation très ancienne; on remplace donc les
#        NaN par la date virtuelle '1800-01-01'
missing_values = df_cap_prod2['date_raccordement'].isna()
df_cap_prod2.loc[missing_values, 'date_raccordement'] = '1800-01-01'

#       Colonne 'date_deraccordement': si non-renseignée, l'installation est supposée active; on remplace donc les NaN 
#        par la date virtuelle '2100-01-01'
missing_values = df_cap_prod2['date_deraccordement'].isna()
df_cap_prod2.loc[missing_values, 'date_deraccordement'] = '2100-01-01'

#       Colonne 'date_mise_en_service'  : si absence, on la suppose identique à la date de date_raccordement
df_cap_prod2.loc[df_cap_prod2['date_mise_en_service'].isna(), 'date_mise_en_service'] = df_cap_prod2['date_raccordement']

In [19]:
#  GESTION DES COLONNES RENSEIGNANT LES COMBUSTIBLES

#          Suppression des colonnes codes, incomplètes et non essentielles au projet
colonnes_to_drop = [9,11,13]
df_cap_prod2.drop(columns = df_cap_prod2.columns[colonnes_to_drop], inplace =True)

#          Gestion des valeurs manquantes des colonnes renseignant les combustibles 
#            Remplacement des NaN des colonnes 'code_EIC','combustible', 'combustibles_secondaires','technologie' et 'type_stockage'
columns_to_replace = ['code_EIC', 'combustible', 'combustibles_secondaires', 'technologie', 'type_stockage']
df_cap_prod2[columns_to_replace] = df_cap_prod2[columns_to_replace].fillna('NR')

#           Remplacement des NaN des colonnes numériques par 0
columns_to_replace2 = ['puis_max_installee','puis_max_rac_charge','puis_max_charge','puis_max_rac','puis_max_installee_discharge',
'energie_stockable','productible','energie_annuelle_glissante_injectee','energie_annuelle_glissante_produite',
'energie_annuelle_glissante_soutiree','energie_annuelle_glissante_stockee']
df_cap_prod2[columns_to_replace2] = df_cap_prod2[columns_to_replace2].fillna(0)

df_cap_prod2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71536 entries, 712 to 73690
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   nomInstallation                      71536 non-null  object        
 1   code_EIC                             71536 non-null  object        
 2   code_region                          71536 non-null  float64       
 3   region                               71536 non-null  object        
 4   date_raccordement                    71536 non-null  datetime64[ns]
 5   date_deraccordement                  71536 non-null  datetime64[ns]
 6   date_mise_en_service                 71536 non-null  datetime64[ns]
 7   code_filiere                         71536 non-null  object        
 8   filiere_production                   71536 non-null  object        
 9   combustible                          71536 non-null  object        
 10  combustibles_

In [20]:
df_cap_prod2.isna().sum()

nomInstallation                        0
code_EIC                               0
code_region                            0
region                                 0
date_raccordement                      0
date_deraccordement                    0
date_mise_en_service                   0
code_filiere                           0
filiere_production                     0
combustible                            0
combustibles_secondaires               0
technologie                            0
type_stockage                          0
puis_max_installee                     0
puis_max_rac_charge                    0
puis_max_charge                        0
puis_max_rac                           0
puis_max_installee_discharge           0
energie_stockable                      0
productible                            0
energie_annuelle_glissante_injectee    0
energie_annuelle_glissante_produite    0
energie_annuelle_glissante_soutiree    0
energie_annuelle_glissante_stockee     0
dtype: int64

In [21]:
# GESTION DES REGIONS

In [22]:
    # Conversion de la colonne 'code_region' en int
df_cap_prod2['code_region'] = df_cap_prod2['code_region'].astype(int)


In [23]:
    # Renommage des noms des régions de la colonne 'region' tels qu'ils le sont dans dataframe Eco2MixRégions traité
dictionnaire_region= {"Provence-Alpes-Côte d'Azur":"PROVENCE ALPES COTE D AZUR",
                      "Bourgogne-Franche-Comté":"BOURGOGNE FRANCHE COMTE",
                      "Bretagne":"BRETAGNE",
                      "Normandie":"NORMANDIE",
                      "Nouvelle-Aquitaine":"NOUVELLE AQUITAINE",
                      "Hauts-de-France":"HAUTS DE FRANCE",
                      "Île-de-France":"ILE DE FRANCE",
                      "Auvergne-Rhône-Alpes":"AUVERGNE RHONE ALPES",
                      "Grand Est":"GRAND EST",
                      "Centre-Val de Loire":"CENTRE VAL DE LOIRE",
                      "Occitanie":"OCCITANIE",
                      "Pays de la Loire":"PAYS DE LA LOIRE"}
df_cap_prod2['region']=df_cap_prod2['region'].replace(dictionnaire_region)

In [24]:
    #  Ajout d'une colonne portant les noms de régions abrégés selon la norme ISO 3166-2
        # Insertion d'une nouvelle colonne 'region_abr' à l'index 3
df_cap_prod2.insert(3, 'region_abr', df_cap_prod2['region'])

In [25]:
        # Affectation des valeurs de 'region_ncc' à 'region_abr'
df_cap_prod2['region_abr'] = df_cap_prod2['region']

In [26]:
        # Renommage des valeurs de 'region_abr'
dictionnaire_region_abr= {"PROVENCE ALPES COTE D AZUR" : "FR-PAC",
                      "BOURGOGNE FRANCHE COMTE" : "FR-BFC",
                      "BRETAGNE" : "FR-BRE",
                      "NORMANDIE" : "FR-NOR",
                      "NOUVELLE AQUITAINE" : "FR-NAQ",
                      "HAUTS DE FRANCE" : "FR-HDF",
                      "ILE DE FRANCE" : "FR-IDF",
                      "AUVERGNE RHONE ALPES":"FR-ARA",
                      "GRAND EST":"FR-GES",
                      "CENTRE VAL DE LOIRE":"FR-CVL",
                      "OCCITANIE":"FR-OCC",
                      "PAYS DE LA LOIRE":"FR-PDL"}
df_cap_prod2['region_abr']=df_cap_prod2['region_abr'].replace(dictionnaire_region_abr)
print("Noms abrégés des régions : ", df_cap_prod2['region_abr'].unique())

Noms abrégés des régions :  ['FR-ARA' 'FR-HDF' 'FR-PAC' 'FR-GES' 'FR-OCC' 'FR-NOR' 'FR-NAQ' 'FR-CVL'
 'FR-BFC' 'FR-BRE' 'FR-PDL' 'FR-IDF']


In [27]:
# GESTION DES NOM DES FILIERES DE PRODUCTION 

In [28]:
print("Valeurs uniques de la colonne 'filiere_production':", df_cap_prod2['filiere_production'].unique())

print("Nombre de valeurs manquantes de la colonne 'filiere_production' :",df_cap_prod2['filiere_production'].isna().sum())
df_cap_prod2.dropna(subset=['filiere_production'], inplace=True)
print("Nombre de valeurs manquantes de la colonne 'filiere_production' :",df_cap_prod2['filiere_production'].isna().sum())

Valeurs uniques de la colonne 'filiere_production': ['Solaire' 'Hydraulique' 'Eolien' 'Thermique non renouvelable'
 'Stockage non hydraulique' 'Bioénergies' 'Nucléaire' 'thermique'
 'stockage_batterie' 'Energies Marines' 'Géothermie' 'eolien']
Nombre de valeurs manquantes de la colonne 'filiere_production' : 0
Nombre de valeurs manquantes de la colonne 'filiere_production' : 0


In [29]:
 # Renommage des valeurs de 'filiere_production' pour équivalence avec les données de 'eco2mix-regional-cons-def'
dictionnaire_region_abr= {'Solaire' :'solaire',
                          'Hydraulique': 'hydraulique',
                          'Eolien' : 'eolien',
                          'Thermique non renouvelable':'thermique',
                          'Stockage non hydraulique' : 'stockage_batterie',
                          'Bioénergies' : 'bioenergies',
                          'Nucléaire' : 'nucleaire',
                          'Energies Marines' : 'hydraulique',
                          'Géothermie':'geothermie'}
df_cap_prod2['filiere_production']=df_cap_prod2['filiere_production'].replace(dictionnaire_region_abr)
print(df_cap_prod2['filiere_production'].unique())

['solaire' 'hydraulique' 'eolien' 'thermique' 'stockage_batterie'
 'bioenergies' 'nucleaire' 'geothermie']


In [30]:
apercu = df_cap_prod2.head(20)

In [31]:
# Export du dataFrame nettoyé dans un fichier au format CSV
df_cap_prod2.to_csv('registre_prod_cleaned.csv', sep=';', encoding='latin-1')

# Export du dataFrame nettoyé dans un fichier Excel

file_export3 ='registre_prod_cleaned.xlsx'

df_cap_prod2.to_excel(file_export3)

print("Registre_national_installation_production_stockage_electricite' nettoyé exporté avec succès dans le fichier Excel 'registre_prod_cleaned.xlsx'")

Registre_national_installation_production_stockage_electricite' nettoyé exporté avec succès dans le fichier Excel 'registre_prod_cleaned.xlsx'


In [32]:
# ======================= CALCUL DES CAPACITES INSTALLEES PAR DATE AVEC PAS DE 00H30, REGION ET FILIERE 
# ================================ de '2013-01-01 00:00:00' à '2022-05-31 23:30:00'

In [33]:
# Création d'un dataframe vide pour le stockage des capacités totales de production
df_result_cap_prod = pd.DataFrame()

import time
# Création d'un générateur pour les dates
def date_range(start, end):
    current = start
    while current < end:
        yield current
        current += pd.Timedelta(minutes=30)

dates = date_range(pd.Timestamp('2013-01-01 00:00:00'), pd.Timestamp('2022-05-31 23:30:00'))

In [ ]:
# Initialisation des variables
dates = list(dates)  # Conversion du générateur en liste
total_iterations = len(dates)  # Nombre total d'itérations
i=0
cumul_temps = 0
start_time = time.time()

for date in dates:
    start_time = time.time()
    
    df_cap_prod2['date'] = date
    df_filtered = df_cap_prod2[(df_cap_prod2['date_mise_en_service'] <= df_cap_prod2['date']) & (df_cap_prod2['date'] < df_cap_prod2['date_deraccordement'])]
    df_temp = df_filtered.groupby(['region', 'filiere_production', 'date'])['puis_max_installee'].sum().reset_index()
    df_result_cap_prod = pd.concat([df_result_cap_prod, df_temp])
    print(i)
     
    end_time = time.time()  # Enregistrement de l'heure de fin
    temps_execution = end_time - start_time  # Calcul du temps d'exécution
    cumul_temps += temps_execution
    i+=1
    print(f"Temps d'exécution de l'itération {i}={temps_execution} secondes.")
    print(f"Temps d'exécution cumulé ={cumul_temps} secondes.")
    
    # Estimation du temps restant
    duree_moy_iteration = cumul_temps / i
    iterations_restantes = total_iterations - i
    temps_restant_estime = duree_moy_iteration * iterations_restantes/60

    print(f"Iteration {i} terminée en {temps_execution} secondes. Temps restant estimé : {temps_restant_estime} minutes.")

0
Temps d'exécution de l'itération 1=0.024932384490966797 secondes.
Temps d'exécution cumulé =0.024932384490966797 secondes.
Iteration 1 terminée en 0.024932384490966797 secondes. Temps restant estimé : 68.57319922447205 minutes.
1
Temps d'exécution de l'itération 2=0.013998031616210938 secondes.
Temps d'exécution cumulé =0.038930416107177734 secondes.
Iteration 2 terminée en 0.013998031616210938 secondes. Temps restant estimé : 53.536134970188144 minutes.
2
Temps d'exécution de l'itération 3=0.01295018196105957 secondes.
Temps d'exécution cumulé =0.051880598068237305 secondes.
Iteration 3 terminée en 0.01295018196105957 secondes. Temps restant estimé : 47.56297940678066 minutes.
3
Temps d'exécution de l'itération 4=0.013969898223876953 secondes.
Temps d'exécution cumulé =0.06585049629211426 secondes.
Iteration 4 terminée en 0.013969898223876953 secondes. Temps restant estimé : 45.277429365118344 minutes.
4
Temps d'exécution de l'itération 5=0.012938499450683594 secondes.
Temps d'exécu

In [ ]:
# Suppression de df_filtered
del df_filtered

# Conversion des valeurs de puissance de kW en MW

df_result_cap_prod['puis_max_installee']=df_result_cap_prod['puis_max_installee'].div(1000)

In [ ]:
# Export du dataFrame nettoyé dans un fichier au format CSV
df_result_cap_prod.to_csv('df_result_cap_prod_2013_2022.csv', sep=';', encoding='latin-1')
print("CONVERSIONS DES KW EN MW EFFECTUEES")
resultats = df_result_cap_prod.head(200)

In [ ]:
df_result_cap_prod.info()

In [ ]:
# Export des 100 premières lignes dans un nouveau fichier CSV pour aperçu
df_result_cap_prod_head = df_result_cap_prod.head(100)
df_result_cap_prod_head.to_csv('df_result_cap_prod_first_100_lines.csv', sep=';', encoding='latin-1', index=False)